In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Integrate and manage Public Transport data").getOrCreate()

In [0]:
spark.conf.set(
    f"fs.azure.account.key.omardbstorageaccount.dfs.core.windows.net", 
    "G3TTTGHer5ZdNR78TdQvASgi9Mw+jDXH8Uk/6558WdDETLX3h3z/yGKvzS4YvfBL5ulvlBcVBvy/+ASt2O0TFA=="
)


In [0]:
import pandas as pd
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

# Define the list of months and the number of days in each month
months = [("January", 31), ("February", 28), ("March", 31), ("April", 30), ("May", 31)]

# Mounting data lake
storageAccountName = "omardbstorageaccount"
storageAccountAccessKey = "G3TTTGHer5ZdNR78TdQvASgi9Mw+jDXH8Uk/6558WdDETLX3h3z/yGKvzS4YvfBL5ulvlBcVBvy/+ASt2O0TFA=="
sasToken = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2023-09-28T20:08:26Z&st=2023-09-26T12:08:26Z&spr=https&sig=7V6Sv9GschHeTzoMg80MHxy1kCl%2FPi7SnVDBozsYMTY%3D"
blobContainerName = "public-transport-data"
mountPoint = "/mnt/public-transport-data/"


if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
    try:
        dbutils.fs.mount(
            source="wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
            mount_point=mountPoint,
            extra_configs={'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
        )
        print("Mount succeeded!")
    except Exception as e:
        print("Mount exception:", e)
else:
    print("Mount point already exists.")

# Generate data for each month
for month_name, num_days in months:
    # Generate data for the specific month
    start_date = datetime(2023, months.index((month_name, num_days)) + 1, 1)
    end_date = start_date + timedelta(days=num_days - 1)
    date_generated = [start_date + timedelta(days=x) for x in range(0, num_days)]

    transport_types = ["Bus", "Train", "Tram", "Metro"]
    routes = ["Route_" + str(i) for i in range(1, 11)]
    stations = ["Station_" + str(i) for i in range(1, 21)]

    # Randomly select 5 days as extreme weather days
    extreme_weather_days = random.sample(date_generated, 5)

    data = []

    for date in date_generated:
        for _ in range(32):  # 32 records per day to get a total of 992 records for each month
            transport = random.choice(transport_types)
            route = random.choice(routes)

            # Normal operating hours
            departure_hour = random.randint(5, 22)
            departure_minute = random.randint(0, 59)

            # Introducing Unusual Operating Hours for buses
            if transport == "Bus" and random.random() < 0.05:  # 5% chance
                departure_hour = 3

            departure_time = f"{departure_hour:02}:{departure_minute:02}"

            # Normal duration
            duration = random.randint(10, 120)

            # Introducing Short Turnarounds
            if random.random() < 0.05:  # 5% chance
                duration = random.randint(1, 5)

            # General delay
            delay = random.randint(0, 15)

            # Weather Impact
            if date in extreme_weather_days:
                # Increase delay by 10 to 60 minutes
                delay += random.randint(10, 60)

                # 10% chance to change the route
                if random.random() < 0.10:
                    route = random.choice(routes)

            total_minutes = departure_minute + duration + delay
            arrival_hour = departure_hour + total_minutes // 60
            arrival_minute = total_minutes % 60

            # Ensure arrival times do not pass 24:59
            if arrival_hour >= 24:
                arrival_hour %= 24

            arrival_time = f"{arrival_hour:02}:{arrival_minute:02}"

            passengers = random.randint(1, 100)
            departure_station = random.choice(stations)
            arrival_station = random.choice(stations)

            data.append([date, transport, route, departure_time, arrival_time, passengers, departure_station, arrival_station, delay])

    # Create a Spark DataFrame from the data
    df = spark.createDataFrame(data, schema=["Date", "TransportType", "Route", "DepartureTime", "ArrivalTime", "Passengers", "DepartureStation", "ArrivalStation", "Delay"])
    
    # Save the DataFrame as a CSV file using the mount point
    file_path = f"/dbfs/mnt/public-transport-data/raw/public_transport_data_{month_name}.csv"
    data = df.toPandas()
    data.to_csv(file_path, index=False)
    print("done")

# Unmount the data lake
dbutils.fs.unmount(mountPoint)
